In [1]:
import os
import rasterio
import numpy as np
import pandas as pd
from tqdm import tqdm  # Import tqdm for progress bars
from rasterio.transform import rowcol, xy

# Define the specific class labels
classes = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220]

# Define directories
mask_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\3. Disturbed Mask\2006 to 2015 Exports\Copernicus LC\Disturbed_Mask_LC"
lst_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\7. LC_LST_Results\4. Compiled_Average_Combination\Corrected\Difference"
output_dir = r"C:\Users\llccl\Downloads\1.Difference"

# Create a mapping from class label to index
class_to_index = {cls: idx for idx, cls in enumerate(classes)}

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Get the list of LST files for progress tracking
lst_files = [f for f in os.listdir(lst_dir) if f.endswith(".tif")]

# Process all LST files with tqdm progress bar
for lst_filename in tqdm(lst_files, desc="Processing LST Files"):
    lst_path = os.path.join(lst_dir, lst_filename)
    output_csv = os.path.join(output_dir, f"{os.path.splitext(lst_filename)[0]}_transition_matrix.csv")
    
    # Initialize a 22x22 matrix to store mean values
    transition_matrix = np.full((22, 22), np.nan)
    
    # Open the LST image
    with rasterio.open(lst_path) as lst_ds:
        lst_data = lst_ds.read(1)
        lst_transform = lst_ds.transform
        lst_crs = lst_ds.crs

        # Iterate over each mask file with a progress bar
        mask_files = [f for f in os.listdir(mask_dir) if f.endswith("_Mask.tif")]
        for filename in tqdm(mask_files, desc=f"Processing Masks for {lst_filename}", leave=False):
            # Parse the class transition from the filename
            parts = filename.split('_')
            class_from = int(parts[0].replace("Class", ""))
            class_to = int(parts[2].replace("Class", ""))
            
            # Check if class_from and class_to are in the specified classes
            if class_from in class_to_index and class_to in class_to_index:
                # Get corresponding indices for matrix placement
                from_idx = class_to_index[class_from]
                to_idx = class_to_index[class_to]
                
                # Open the mask image
                mask_path = os.path.join(mask_dir, filename)
                with rasterio.open(mask_path) as mask_ds:
                    mask_data = mask_ds.read(1)
                    mask_transform = mask_ds.transform
                    mask_crs = mask_ds.crs
                    
                    # Check if CRS matches
                    if mask_crs != lst_crs:
                        print(f"CRS mismatch: {mask_crs} (mask) vs {lst_crs} (LST). Skipping {filename}.")
                        continue
                    
                    # Find coordinates of mask pixels with value 1
                    mask_indices = np.argwhere(mask_data == 1)
                    mask_coords = [xy(mask_transform, row, col) for row, col in mask_indices]
                    
                    # Map mask coordinates to LST pixel indices
                    lst_indices = [rowcol(lst_transform, x, y) for x, y in mask_coords]
                    lst_values = []
                    
                    for row, col in lst_indices:
                        if 0 <= row < lst_data.shape[0] and 0 <= col < lst_data.shape[1]:
                            value = lst_data[row, col]
                            if not np.isnan(value):  # Exclude NaN values
                                lst_values.append(value)
                    
                    # Remove NaN and calculate statistics
                    lst_values = np.array(lst_values)
                    if lst_values.size > 0:
                        q1 = np.percentile(lst_values, 25)
                        q3 = np.percentile(lst_values, 75)
                        filtered_values = lst_values[(lst_values >= q1) & (lst_values <= q3)]
                        
                        if filtered_values.size > 0:
                            mean_value = np.mean(filtered_values)
                            transition_matrix[from_idx, to_idx] = mean_value
    
    # Convert matrix to DataFrame and save to CSV
    transition_df = pd.DataFrame(transition_matrix, columns=[f'Class{cls}' for cls in classes], index=[f'Class{cls}' for cls in classes])
    transition_df.to_csv(output_csv, index=True)
    print(f"Processed {lst_filename} and saved transition matrix to {output_csv}")

Processing LST Files:   0%|                                                                      | 0/4 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_Average.tif:   0%|   | 0/484 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_Average.tif:   0%| | 1/484 [00:50<6:47:39, 50
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_Average.tif:   0%| | 2/484 [01:39<6:36:31, 49
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_Average.tif:   1%| | 3/484 [02:31<6:47:13, 50
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_Average.tif:   1%| | 4/484 [03:36<7:31:22, 56
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_Average.tif:   1%| | 5/484 [04:32<7:29:14, 56
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_Average.tif:   1%| | 6/484 [05:21<7:09:00, 53
Processing Masks for reprojected_correct

Processed reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\1.Difference\reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_Average_transition_matrix.csv



Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif:   0%| | 0/484 [00:00<
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif:   0%| | 1/484 [00:43<
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif:   0%| | 2/484 [01:23<
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif:   1%| | 3/484 [02:03<
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif:   1%| | 4/484 [02:51<
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif:   1%| | 5/484 [03:33<
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif:   1%| | 6/484 [04:09<
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif:   1%| | 7/484 [04:48<
Processing Masks for reprojected_correc

Processed reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif and saved transition matrix to C:\Users\llccl\Downloads\1.Difference\reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination_transition_matrix.csv



Processing Masks for reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average.tif:   0%| | 0/484 [00:00<?, ?it/
Processing Masks for reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average.tif:   0%| | 1/484 [00:51<6:50:56
Processing Masks for reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average.tif:   0%| | 2/484 [01:32<6:03:01
Processing Masks for reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average.tif:   1%| | 3/484 [02:10<5:38:56
Processing Masks for reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average.tif:   1%| | 4/484 [02:57<5:50:19
Processing Masks for reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average.tif:   1%| | 5/484 [03:38<5:43:38
Processing Masks for reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average.tif:   1%| | 6/484 [04:14<5:24:14
Processing Masks for reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average.tif:   1%| | 7/484 [04:53<5:17:26
Processing Masks for reprojected_correc

Processed reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\1.Difference\reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average_transition_matrix.csv



Processing Masks for reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average.tif:   0%| | 0/484 [00:00<?, ?it/
Processing Masks for reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average.tif:   0%| | 1/484 [00:43<5:53:49
Processing Masks for reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average.tif:   0%| | 2/484 [01:24<5:37:35
Processing Masks for reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average.tif:   1%| | 3/484 [02:04<5:27:14
Processing Masks for reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average.tif:   1%| | 4/484 [02:54<5:56:34
Processing Masks for reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average.tif:   1%| | 5/484 [03:34<5:44:17
Processing Masks for reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average.tif:   1%| | 6/484 [04:10<5:24:14
Processing Masks for reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average.tif:   1%| | 7/484 [04:49<5:17:57
Processing Masks for reprojected_correc

Processed reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\1.Difference\reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average_transition_matrix.csv


In [1]:
import os
import rasterio
import numpy as np
import pandas as pd
from tqdm import tqdm  # Import tqdm for progress bars
from rasterio.transform import rowcol, xy

# Define the specific class labels
classes = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220]

# Define directories
mask_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\3. Disturbed Mask\2006 to 2015 Exports\Copernicus LC\Disturbed_Mask_LC"
lst_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\7. LC_LST_Results\4. Compiled_Average_Combination\Corrected\Linear Trend"
output_dir = r"C:\Users\llccl\Downloads\2.LinearTrend"

# Create a mapping from class label to index
class_to_index = {cls: idx for idx, cls in enumerate(classes)}

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Get the list of LST files for progress tracking
lst_files = [f for f in os.listdir(lst_dir) if f.endswith(".tif")]

# Process all LST files with tqdm progress bar
for lst_filename in tqdm(lst_files, desc="Processing LST Files"):
    lst_path = os.path.join(lst_dir, lst_filename)
    output_csv = os.path.join(output_dir, f"{os.path.splitext(lst_filename)[0]}_transition_matrix.csv")
    
    # Initialize a 22x22 matrix to store mean values
    transition_matrix = np.full((22, 22), np.nan)
    
    # Open the LST image
    with rasterio.open(lst_path) as lst_ds:
        lst_data = lst_ds.read(1)
        lst_transform = lst_ds.transform
        lst_crs = lst_ds.crs

        # Iterate over each mask file with a progress bar
        mask_files = [f for f in os.listdir(mask_dir) if f.endswith("_Mask.tif")]
        for filename in tqdm(mask_files, desc=f"Processing Masks for {lst_filename}", leave=False):
            # Parse the class transition from the filename
            parts = filename.split('_')
            class_from = int(parts[0].replace("Class", ""))
            class_to = int(parts[2].replace("Class", ""))
            
            # Check if class_from and class_to are in the specified classes
            if class_from in class_to_index and class_to in class_to_index:
                # Get corresponding indices for matrix placement
                from_idx = class_to_index[class_from]
                to_idx = class_to_index[class_to]
                
                # Open the mask image
                mask_path = os.path.join(mask_dir, filename)
                with rasterio.open(mask_path) as mask_ds:
                    mask_data = mask_ds.read(1)
                    mask_transform = mask_ds.transform
                    mask_crs = mask_ds.crs
                    
                    # Check if CRS matches
                    if mask_crs != lst_crs:
                        print(f"CRS mismatch: {mask_crs} (mask) vs {lst_crs} (LST). Skipping {filename}.")
                        continue
                    
                    # Find coordinates of mask pixels with value 1
                    mask_indices = np.argwhere(mask_data == 1)
                    mask_coords = [xy(mask_transform, row, col) for row, col in mask_indices]
                    
                    # Map mask coordinates to LST pixel indices
                    lst_indices = [rowcol(lst_transform, x, y) for x, y in mask_coords]
                    lst_values = []
                    
                    for row, col in lst_indices:
                        if 0 <= row < lst_data.shape[0] and 0 <= col < lst_data.shape[1]:
                            value = lst_data[row, col]
                            if not np.isnan(value):  # Exclude NaN values
                                lst_values.append(value)
                    
                    # Remove NaN and calculate statistics
                    lst_values = np.array(lst_values)
                    if lst_values.size > 0:
                        q1 = np.percentile(lst_values, 25)
                        q3 = np.percentile(lst_values, 75)
                        filtered_values = lst_values[(lst_values >= q1) & (lst_values <= q3)]
                        
                        if filtered_values.size > 0:
                            mean_value = np.mean(filtered_values)
                            transition_matrix[from_idx, to_idx] = mean_value
    
    # Convert matrix to DataFrame and save to CSV
    transition_df = pd.DataFrame(transition_matrix, columns=[f'Class{cls}' for cls in classes], index=[f'Class{cls}' for cls in classes])
    transition_df.to_csv(output_csv, index=True)
    print(f"Processed {lst_filename} and saved transition matrix to {output_csv}")

Processing LST Files:   0%|                                                                      | 0/4 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_Average.tif:   0%| | 0/484 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_Average.tif:   0%| | 1/484 [01:01<8:15:28, 
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_Average.tif:   0%| | 2/484 [02:14<9:06:34, 
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_Average.tif:   1%| | 3/484 [03:21<9:01:10, 
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_Average.tif:   1%| | 4/484 [04:12<8:08:36, 
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_Average.tif:   1%| | 5/484 [04:57<7:21:34, 
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_Average.tif:   1%| | 6/484 [05:41<6:49:59, 
Processing Masks for reprojected_correct

Processed reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\2.LinearTrend\reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_Average_transition_matrix.csv



Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif:   0%| | 0/484 [00:0
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif:   0%| | 1/484 [00:5
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif:   0%| | 2/484 [01:3
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif:   1%| | 3/484 [02:2
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif:   1%| | 4/484 [03:2
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif:   1%| | 5/484 [04:0
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif:   1%| | 6/484 [04:5
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif:   1%| | 7/484 [05:3
Processing Masks for reprojected_correc

Processed reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif and saved transition matrix to C:\Users\llccl\Downloads\2.LinearTrend\reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination_transition_matrix.csv



Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average.tif:   0%|  | 0/484 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average.tif:   0%| | 1/484 [00:42<5:44:47, 4
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average.tif:   0%| | 2/484 [01:21<5:25:09, 4
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average.tif:   1%| | 3/484 [02:01<5:22:12, 4
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average.tif:   1%| | 4/484 [02:50<5:47:54, 4
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average.tif:   1%| | 5/484 [03:31<5:41:40, 4
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average.tif:   1%| | 6/484 [04:08<5:25:14, 4
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average.tif:   1%| | 7/484 [04:46<5:16:48, 3
Processing Masks for reprojected_correc

Processed reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\2.LinearTrend\reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average_transition_matrix.csv



Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average.tif:   0%|  | 0/484 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average.tif:   0%| | 1/484 [00:44<5:58:26, 4
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average.tif:   0%| | 2/484 [01:24<5:35:29, 4
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average.tif:   1%| | 3/484 [02:03<5:23:37, 4
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average.tif:   1%| | 4/484 [02:50<5:46:04, 4
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average.tif:   1%| | 5/484 [03:32<5:41:59, 4
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average.tif:   1%| | 6/484 [04:10<5:27:18, 4
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average.tif:   1%| | 7/484 [04:48<5:18:00, 4
Processing Masks for reprojected_correc

Processed reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\2.LinearTrend\reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average_transition_matrix.csv


In [2]:
import os
import rasterio
import numpy as np
import pandas as pd
from tqdm import tqdm  # Import tqdm for progress bars
from rasterio.transform import rowcol, xy

# Define the specific class labels
classes = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220]

# Define directories
mask_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\3. Disturbed Mask\2006 to 2015 Exports\Copernicus LC\Disturbed_Mask_LC"
lst_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\7. LC_LST_Results\4. Compiled_Average_Combination\Corrected\Mean"
output_dir = r"C:\Users\llccl\Downloads\3.Mean"

# Create a mapping from class label to index
class_to_index = {cls: idx for idx, cls in enumerate(classes)}

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Get the list of LST files for progress tracking
lst_files = [f for f in os.listdir(lst_dir) if f.endswith(".tif")]

# Process all LST files with tqdm progress bar
for lst_filename in tqdm(lst_files, desc="Processing LST Files"):
    lst_path = os.path.join(lst_dir, lst_filename)
    output_csv = os.path.join(output_dir, f"{os.path.splitext(lst_filename)[0]}_transition_matrix.csv")
    
    # Initialize a 22x22 matrix to store mean values
    transition_matrix = np.full((22, 22), np.nan)
    
    # Open the LST image
    with rasterio.open(lst_path) as lst_ds:
        lst_data = lst_ds.read(1)
        lst_transform = lst_ds.transform
        lst_crs = lst_ds.crs

        # Iterate over each mask file with a progress bar
        mask_files = [f for f in os.listdir(mask_dir) if f.endswith("_Mask.tif")]
        for filename in tqdm(mask_files, desc=f"Processing Masks for {lst_filename}", leave=False):
            # Parse the class transition from the filename
            parts = filename.split('_')
            class_from = int(parts[0].replace("Class", ""))
            class_to = int(parts[2].replace("Class", ""))
            
            # Check if class_from and class_to are in the specified classes
            if class_from in class_to_index and class_to in class_to_index:
                # Get corresponding indices for matrix placement
                from_idx = class_to_index[class_from]
                to_idx = class_to_index[class_to]
                
                # Open the mask image
                mask_path = os.path.join(mask_dir, filename)
                with rasterio.open(mask_path) as mask_ds:
                    mask_data = mask_ds.read(1)
                    mask_transform = mask_ds.transform
                    mask_crs = mask_ds.crs
                    
                    # Check if CRS matches
                    if mask_crs != lst_crs:
                        print(f"CRS mismatch: {mask_crs} (mask) vs {lst_crs} (LST). Skipping {filename}.")
                        continue
                    
                    # Find coordinates of mask pixels with value 1
                    mask_indices = np.argwhere(mask_data == 1)
                    mask_coords = [xy(mask_transform, row, col) for row, col in mask_indices]
                    
                    # Map mask coordinates to LST pixel indices
                    lst_indices = [rowcol(lst_transform, x, y) for x, y in mask_coords]
                    lst_values = []
                    
                    for row, col in lst_indices:
                        if 0 <= row < lst_data.shape[0] and 0 <= col < lst_data.shape[1]:
                            value = lst_data[row, col]
                            if not np.isnan(value):  # Exclude NaN values
                                lst_values.append(value)
                    
                    # Remove NaN and calculate statistics
                    lst_values = np.array(lst_values)
                    if lst_values.size > 0:
                        q1 = np.percentile(lst_values, 25)
                        q3 = np.percentile(lst_values, 75)
                        filtered_values = lst_values[(lst_values >= q1) & (lst_values <= q3)]
                        
                        if filtered_values.size > 0:
                            mean_value = np.mean(filtered_values)
                            transition_matrix[from_idx, to_idx] = mean_value
    
    # Convert matrix to DataFrame and save to CSV
    transition_df = pd.DataFrame(transition_matrix, columns=[f'Class{cls}' for cls in classes], index=[f'Class{cls}' for cls in classes])
    transition_df.to_csv(output_csv, index=True)
    print(f"Processed {lst_filename} and saved transition matrix to {output_csv}")

Processing LST Files:   0%|                                                                      | 0/4 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_Average.tif:   0%|        | 0/484 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_Average.tif:   0%| | 1/484 [00:44<6:01:04, 44.85s/
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_Average.tif:   0%| | 2/484 [01:27<5:47:32, 43.26s/
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_Average.tif:   1%| | 3/484 [02:08<5:38:49, 42.26s/
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_Average.tif:   1%| | 4/484 [03:00<6:09:58, 46.25s/
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_Average.tif:   1%| | 5/484 [03:50<6:21:04, 47.73s/
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_Average.tif:   1%| | 6/484 [04:33<6:06:01, 45.94s/
Processing Masks for reprojected_correct

Processed reprojected_corrected_MOYD21A1DN_2000-2020_mean_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\3.Mean\reprojected_corrected_MOYD21A1DN_2000-2020_mean_Average_transition_matrix.csv



Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination.tif:   0%| | 0/484 [00:00<?, ?i
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination.tif:   0%| | 1/484 [00:42<5:38:
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination.tif:   0%| | 2/484 [01:22<5:28:
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination.tif:   1%| | 3/484 [02:01<5:22:
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination.tif:   1%| | 4/484 [02:51<5:51:
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination.tif:   1%| | 5/484 [03:33<5:46:
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination.tif:   1%| | 6/484 [04:12<5:32:
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination.tif:   1%| | 7/484 [04:51<5:25:
Processing Masks for reprojected_correc

Processed reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination.tif and saved transition matrix to C:\Users\llccl\Downloads\3.Mean\reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination_transition_matrix.csv



Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_mean_Average.tif:   0%|         | 0/484 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_mean_Average.tif:   0%| | 1/484 [00:43<5:53:36, 43.93s/i
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_mean_Average.tif:   0%| | 2/484 [01:28<5:55:40, 44.28s/i
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_mean_Average.tif:   1%| | 3/484 [02:14<5:59:40, 44.87s/i
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_mean_Average.tif:   1%| | 4/484 [03:03<6:13:05, 46.64s/i
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_mean_Average.tif:   1%| | 5/484 [03:44<5:55:29, 44.53s/i
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_mean_Average.tif:   1%| | 6/484 [04:22<5:36:44, 42.27s/i
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_mean_Average.tif:   1%| | 7/484 [05:04<5:35:48, 42.24s/i
Processing Masks for reprojected_correc

Processed reprojected_corrected_MOYD21A1D_2000-2020_mean_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\3.Mean\reprojected_corrected_MOYD21A1D_2000-2020_mean_Average_transition_matrix.csv



Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_mean_Average.tif:   0%|         | 0/484 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_mean_Average.tif:   0%| | 1/484 [00:42<5:39:36, 42.19s/i
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_mean_Average.tif:   0%| | 2/484 [01:22<5:28:58, 40.95s/i
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_mean_Average.tif:   1%| | 3/484 [01:59<5:16:13, 39.45s/i
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_mean_Average.tif:   1%| | 4/484 [02:46<5:38:34, 42.32s/i
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_mean_Average.tif:   1%| | 5/484 [03:34<5:55:11, 44.49s/i
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_mean_Average.tif:   1%| | 6/484 [04:11<5:32:20, 41.72s/i
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_mean_Average.tif:   1%| | 7/484 [04:49<5:21:52, 40.49s/i
Processing Masks for reprojected_correc

Processed reprojected_corrected_MOYD21A1N_2000-2020_mean_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\3.Mean\reprojected_corrected_MOYD21A1N_2000-2020_mean_Average_transition_matrix.csv
